In [1]:
# importing libraries
import os
from dotenv import load_dotenv
load_dotenv()
import requests
import json
import pandas as pd

In [2]:
# get access to the token we have saved in our .env
api_key = os.getenv('token')

In [3]:
# get parameters
parameters = {"Auhorization" : f"token {api_key}"}

In [4]:
# read our dataset - outliers
out = pd.read_csv('gbif_out.csv')

In [5]:
# create a list of species
spec = list(out.species.unique())
type(spec)

list

In [6]:
# call the IUCN api in order to get the Animal Threat Category
# we are also interested in the common name of every specie and the population trend

def spec_info(df, token):
    result = []
    for d in df:
        try:
            gen = d.split()[0].lower()
            sp = d.split()[1]
            url = f"https://apiv3.iucnredlist.org/api/v3/species/{gen}%20{sp}?token={api_key}"
            get = requests.get(url)
            info = get.json()

            result.append(info.get('result')[0])
        except:
            result.append({'taxonid': 'NaN', 
                           'scientific_name': f'{d}',
                           'kingdom': 'ANIMALIA',
                           'phylum': 'CHORDATA',
                           'class': 'AVES', 
                           'order': 'NaN',
                           'family': 'NaN',
                           'genus': 'NaN',
                           'main_common_name': 'NaN',
                           'authority': 'NaN',
                           'published_year': 'NaN',
                           'assessment_date': 'NaN',
                           'category': 'NaN', 
                           'criteria': 'NaN', 
                           'population_trend': 'NaN', 
                           'marine_system': 'NaN', 
                           'freshwater_system': 'NaN', 
                           'terrestrial_system': 'NaN', 
                           'assessor': 'NaN', 
                           'reviewer': 'NaN',
                           'eoo_km2': 'NaN',
                           'elevation_upper': 'NaN',
                           'elevation_lower': 'NaN',
                           'depth_upper': 'NaN',
                           'depth_lower': 'NaN',
                           'errata_flag': 'NaN',
                           'errata_reason': 'NaN',
                           'amended_flag': 'NaN',
                           'amended_reason': 'NaN'})
            
    return pd.DataFrame.from_dict(result)


In [7]:
iucn = spec_info(df = spec, token = api_key)

In [8]:
# create dics with relevant info 
category = dict(zip(iucn.scientific_name, iucn.category))
common_name = dict(zip(iucn.scientific_name, iucn.main_common_name))
population_trend = dict(zip(iucn.scientific_name, iucn.population_trend))


# use map to apply dics to each specie
out['category'] = out.species.map(category)
out['common_name'] = out.species.map(common_name)
out['population_trend'] = out.species.map(population_trend)


In [9]:
# save outliers in dataset
out.to_csv('outliers.csv')

- Extinct **(EX)** – beyond reasonable doubt that the species is no longer extant.
- Extinct in the wild **(EW)** – survives only in captivity, cultivation and/or outside native range, as presumed after exhaustive surveys.
- Critically endangered **(CR)** – in a particularly and extremely critical state.
- Endangered **(EN)** – very high risk of extinction in the wild, meets any of criteria A to E for Endangered.
- Vulnerable **(VU)** – meets one of the 5 red list criteria and thus considered to be at high risk of unnatural (human-caused) extinction without further human intervention.
- Near threatened **(NT)** – close to being at high risk of extinction in the near future.
- Least concern **(LC)** – unlikely to become extinct in the near future.
- Data deficient **(DD)**
- Not evaluated **(NE)**

In [10]:
out.category.unique()

array(['NT', 'LC', 'VU', 'NaN'], dtype=object)

In [11]:
# found some empty values (5 species in 35) we don't have information to
vacios = out.loc[:, "category"] == 'NaN'
empty = out[vacios]
to_drop = list(empty.species.unique())

In [12]:
# drop species with empty values returned from the call to the api
out = out[~out['species'].isin(to_drop)]
len(out.species.unique())

30

In [13]:
# Least concern (LC) – unlikely to become extinct in the near future
LC = out.loc[:, 'category'] == 'LC'
print(out[LC].species.unique())

['Tadorna ferruginea' 'Passer hispaniolensis' 'Podiceps grisegena'
 'Fulica cristata' 'Alopochen aegyptiaca' 'Morus bassanus'
 'Sylvia conspicillata' 'Sterna hirundo' 'Chlidonias hybrida'
 'Phasianus colchicus' 'Calidris temminckii' 'Sterna paradisaea'
 'Anser brachyrhynchus' 'Phylloscopus inornatus'
 'Stercorarius longicaudus' 'Threskiornis aethiopicus' 'Tringa flavipes'
 'Xema sabini' 'Larus hyperboreus' 'Alle alle' 'Fulmarus glacialis'
 'Aix galericulata' 'Sylvia curruca' 'Anthus hodgsoni' 'Anser fabalis'
 'Phylloscopus fuscatus' 'Aix sponsa']


In [14]:
# Near threatened (NT) – close to being at high risk of extinction in the near future.
NT = out.loc[:, 'category'] == 'NT'
print(out[NT].species.unique())

['Phoeniconaias minor' 'Calidris canutus']


In [15]:
# Vulnerable (VU) considered to be at high risk of unnatural (human-caused) extinction
VU = out.loc[:, 'category'] == 'VU'
print(out[VU].species.unique())

['Marmaronetta angustirostris']


In [16]:
# further study about the population trend
out.population_trend.unique()

array(['Decreasing', 'Unknown', 'Increasing', 'Stable'], dtype=object)

In [17]:
# 10 more species (+3 in VU and NT) population are decreasing
dec = out.loc[:, 'population_trend'] == 'Decreasing'
print(out[dec].species.unique())

['Phoeniconaias minor' 'Passer hispaniolensis' 'Calidris canutus'
 'Podiceps grisegena' 'Marmaronetta angustirostris' 'Fulica cristata'
 'Alopochen aegyptiaca' 'Phasianus colchicus' 'Sterna paradisaea'
 'Tringa flavipes' 'Alle alle' 'Aix galericulata' 'Anser fabalis']


In [18]:
dec = out[dec]

# Passer hispaniolensis
La especie no presenta problemas de conservación; por el contrario, la gran expansión experimentada en la cuenca del Guadiana ha hecho que llegue a considerársela una plaga en las zonas de cultivos de regadío de Extremadura, donde se llevan a cabo ocasionales descastes de población y medidas encaminadas a evitar el asentamiento de colonias en las cercanías de los cultivos. Puede, no obstante, verse algo afectada por la pérdida de
arboledas donde criar, especialmente en zonas periféricas de su área de distribución

-- SEO BIRDLIFE


# Phasianus colchicus
Dada su estrecha ligazón con la actividad cinegética, su principal factor limitante es la fuerte presión de caza que sufre. No se ha evaluado convenientemente la posible incidencia de su liberación en el medio natural (impacto sobre la vegetación, competencia con otras especies, problemas
sanitarios, predadores, etc.).

-- SEO BIRDLIFE

In [19]:
no = ['Passer hispaniolensis', 'Phasianus colchicus']

decreasing = dec[~dec['species'].isin(no)]
decreasing.species.unique()

array(['Phoeniconaias minor', 'Calidris canutus', 'Podiceps grisegena',
       'Marmaronetta angustirostris', 'Fulica cristata',
       'Alopochen aegyptiaca', 'Sterna paradisaea', 'Tringa flavipes',
       'Alle alle', 'Aix galericulata', 'Anser fabalis'], dtype=object)

In [20]:
# save decreasing species in dataset
decreasing.to_csv('decreasing.csv')